### Set the Workspace

In [ ]:
from azureml.core import Workspace

# Leer Workspace directamente desde archivo
ws = Workspace.from_config(path="id_CONFIDENTIAL.json")

### Upload the Model

In [20]:
from azureml.core.model import Model

registered_model = Model.register(model_path="modelo_regresion.sav",
                                  model_name="modelo_random_forest_reg",
                                  workspace=ws)

Registering model modelo_random_forest_reg


### Environment Configuration

In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) 


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py", 
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\david\AppData\Local\Temp\ipykernel_21180\944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


### Deployment

In [22]:
import json
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})

file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
